In [1]:
import re

import pandas as pd

from tools import criaChaveAcordao
from sklearn import preprocessing

In [ ]:
# # Transforma o texto em string; letras maiúsculas; remove espaços nas laterais da strig
# def padronizaTexto(texto):
#     return str(texto.upper().strip())


# # Função que gera a chave para os acordãos, criada para ser aplicada na coluna "Título" do dataset positivo
# def criaChaveAcordao(texto):
    
#     # Atribui valor inicil para a variável indicadors do tipo da sessão
#     tipoSessao = 'PL'
    
#     # Aplica função de padronizar o dado de entrada
#     texto = padronizaTexto(texto)
 
#     # Extraindo os valores para variável tipoDecisao:        
#     if len(re.findall('AC[O|Ó]RD[Ã|A]O\s[A-Z]',texto)) > 0:
#         tipoDecisao = 'AR'
#     elif len(re.findall('DECIS[Ã|A]O',texto)) > 0:
#         tipoDecisao = 'DE'
#     else:
#         tipoDecisao = 'AC'
    
#     # Extraindo os valores para as variáveis nrAcordao e ano:    
#     ## Procura no texto todos os números e gera uma lista
#     anoAcordao = re.findall("[0-9]+",texto)
    
#     if len(anoAcordao[1]) < 4:
#         if int(anoAcordao[1]) <= 30:
#             anoAcordao[1] = '20' + anoAcordao[1]
#         else:
#             anoAcordao[1] = '19' + anoAcordao[1]

#     # Extraindo os valores para variável tipoSessao:
    
#     ### Atribui valores à variável de acordo com o tipo de sessão. Caso seja PLENÁRIO, a variável tiposessão recebe valor "PL";
#     ### caso seja PRIMEIRA CÂMARA, a variável tiposessão recebe valor "1C";
#     ### caso seja SEGUNDA CÂMARA, a variável tiposessão recebe valor "2C"
#     if(len(re.findall('PLEN[Á|A]RIO',texto)) == 0):
#         if(len(re.findall('PRIMEIRA',texto)) != 0):
#             tipoSessao = '1C'
#         elif(len(re.findall('SEGUNDA',texto)) > 0):
#             tipoSessao = '2C'
 
#     return tipoDecisao + '-' + anoAcordao[0].zfill(6) + '-' + anoAcordao[1] + '-' + tipoSessao

## Conhecer a classe positiva

Os acórdãos da classe positiva foram encontrados a partir de pesquisa no sitio eletrônico do TCU (portal de jurisprudência), por meio de pesquisa de termos chave que fazem referência à tipologias de cláusulas restritivas de editais de licitação (cf. NAVES, Fernanda de Moura Ribeiro. TCU e stare decisis administrativo: o caso das cláusulas restritivas em editais de licitação de obras de engenharia. Dissertação (Mestrado Profissional em Direito e Políticas Públicas) - Universidade Federal e Goiás, 2019, p. 83-88).
<https://pesquisa.apps.tcu.gov.br/#/pesquisa/acordao-completo>. Acesso em 18 out 2019.

In [3]:
df_positivo = pd.read_excel('dados/acordaopositivo.xlsx', sheet_name='Sheet1')
df_positivo.insert(0, 'ID', '')

## Conhecer a classe negativa

Os acórdãos da classe negativa foram obtidos a partir de pesquisa na jurisprudência selecionada do TCU. O TCU classifica aproximadamente 14 mil acórdãos em 10 áreas, 393 temas e 817 subtemas. Excluiu-se temas e subtemas que poderiam estar associados à "cláusulas restritivas de editais de licitação", restando 10.688 acórdãos previamente classificados pelo TCu e que, em tese, não guardam relação com a temática em estudo, sendo, por isso, utilizados como a classe negativa.
<https://pesquisa.apps.tcu.gov.br/#/pesquisa/jurisprudencia-selecionada>. Acesso em 18 out 2019.

In [4]:
df_negativo = pd.read_excel('dados/acordaonegativo.xlsx', sheet_name='Sheet1')
df_negativo.insert(0, 'ID', '')

In [5]:
print(df_positivo.info(), '\n')
print(df_negativo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 16 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   ID                                      634 non-null    object
 1   LabelClasse                             634 non-null    object
 2   Tipo                                    634 non-null    object
 3   Título                                  634 non-null    object
 4   Data                                    634 non-null    object
 5   Relator                                 634 non-null    object
 6   Sumário                                 626 non-null    object
 7   Processo                                634 non-null    object
 8   Tipo de processo                        634 non-null    object
 9   Interessado / Responsável / Recorrente  630 non-null    object
 10  Entidade                                573 non-null    object
 11  Repres

In [6]:
df_positivo.rename(str.upper, axis='columns', inplace=True)
df_positivo.rename(columns={'LABELCLASSE': 'CLASSE', 'TÍTULO': 'TITULO', 'SUMÁRIO': 'SUMARIO', 
                            'TIPO DE PROCESSO': 'TIPO_PROCESSO',
                           'INTERESSADO / RESPONSÁVEL / RECORRENTE': 'INTERESSADO', 
                           'REPRESENTANTE DO MINISTÉRIO PÚBLICO': 'REP_MP',
                           'UNIDADE TÉCNICA': 'UNIDADE_TECNICA', 'REPRESENTANTE LEGAL': 'REP_LEGAL',
                           'ENDEREÇO DO ARQUIVO': 'FILE_PATH'}, inplace=True)


In [7]:
df_negativo.rename(str.upper, axis='columns', inplace=True)
df_negativo.rename(columns={'ACÓRDÃO': 'ACORDAO', 'ÁREA':'AREA', 
                            'TIPO DO PROCESSO': 'TIPO_PROCESSO'}, inplace=True)


In [8]:
print(df_positivo.info(), '\n')
print(df_negativo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               634 non-null    object
 1   CLASSE           634 non-null    object
 2   TIPO             634 non-null    object
 3   TITULO           634 non-null    object
 4   DATA             634 non-null    object
 5   RELATOR          634 non-null    object
 6   SUMARIO          626 non-null    object
 7   PROCESSO         634 non-null    object
 8   TIPO_PROCESSO    634 non-null    object
 9   INTERESSADO      630 non-null    object
 10  ENTIDADE         573 non-null    object
 11  REP_MP           452 non-null    object
 12  UNIDADE_TECNICA  614 non-null    object
 13  REP_LEGAL        534 non-null    object
 14  ASSUNTO          306 non-null    object
 15  FILE_PATH        634 non-null    object
dtypes: object(16)
memory usage: 79.4+ KB
None 

<class 'pandas.core.frame.DataFrame'

In [9]:
# Converte a classe em categorias
for i in range(0,14):
    df_positivo.loc[df_positivo['CLASSE'] == 'classe' + str(i+1), ['CLASSE']] = i+1

In [10]:
df_positivo.CLASSE.value_counts()

13    107
5      96
14     82
8      57
9      49
11     46
7      46
10     41
6      25
12     22
4      21
3      19
2      17
1       6
Name: CLASSE, dtype: int64

In [11]:
# Insere o valor do campo ID
for index, row in df_positivo.iterrows():
    df_positivo.iloc[index, 0] = criaChaveAcordao(df_positivo.iloc[index].TITULO)
    
df_positivo.insert(1, 'LABEL', 1)

In [12]:
# Insere o valor do campo ID
for index, row in df_negativo.iterrows():
    df_negativo.iloc[index, 0] = criaChaveAcordao(df_negativo.iloc[index].ACORDAO)
    
df_negativo.insert(1, 'LABEL', 0)

In [13]:
df_positivo.to_csv('dados/positivos.csv', sep='|', encoding='utf-8', index=False)
df_negativo.to_csv('dados/negativos.csv', sep='|', encoding='utf-8', index=False)

In [14]:
df_positivo.TIPO.value_counts()

ACÓRDÃO               609
DECISÃO                19
ACÓRDÃO DE RELAÇÃO      6
Name: TIPO, dtype: int64

In [15]:
print(df_negativo.TIPO_PROCESSO.value_counts().shape[0])
print(df_positivo.TIPO_PROCESSO.value_counts().shape[0])

32
17


In [16]:
print(df_positivo.TIPO_PROCESSO.value_counts(), '\n')
print(df_negativo.TIPO_PROCESSO.value_counts())

REPRESENTAÇÃO (REPR)                        323
RELATÓRIO DE AUDITORIA (RA)                 175
RELATÓRIO DE LEVANTAMENTO (RL)               49
TOMADA DE CONTAS ESPECIAL (TCE)              21
DENÚNCIA (DEN)                               13
SOLICITAÇÃO DO CONGRESSO NACIONAL (SCN)       7
TOMADA DE CONTAS SIMPLIFICADA (TCSP)          7
MONITORAMENTO (MON)                           6
PRESTAÇÃO DE CONTAS (PC)                      5
RELATÓRIO DE MONITORAMENTO (RMON)             5
PRESTAÇÃO DE CONTAS SIMPLIFICADA (PCSP)       5
DESESTATIZAÇÃO (DES)                          5
ACOMPANHAMENTO (ACOM)                         4
RELATÓRIO DE ACOMPANHAMENTO (RACOM)           3
CONSULTA (CONS)                               3
CONTAS DO PRESIDENTE DA REPÚBLICA (CGOV)      2
ADMINISTRATIVO (ADM)                          1
Name: TIPO_PROCESSO, dtype: int64 

TOMADA DE CONTAS ESPECIAL                             3344
APOSENTADORIA                                         1665
REPRESENTAÇÃO                 

In [17]:
pd.merge(df_positivo, df_negativo, on=['TIPO_PROCESSO'], how='inner')

,ID_x,LABEL_x,CLASSE,TIPO,TITULO,DATA,RELATOR,SUMARIO,PROCESSO,TIPO_PROCESSO,INTERESSADO,ENTIDADE,REP_MP,UNIDADE_TECNICA,REP_LEGAL,ASSUNTO,FILE_PATH,ID_y,LABEL_y,ACORDAO,AREA,TEMA,SUBTEMA,ENUNCIADO


In [18]:
set(df_positivo.TIPO_PROCESSO).intersection(set(df_negativo.TIPO_PROCESSO))

set()